In [16]:
# **IMPORTANDO LIBRERIAS**

In [17]:
import sys
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy import create_engine, Column, Integer, String, VARCHAR,Boolean, TIMESTAMP
from sqlalchemy.types import DateTime
from sqlalchemy.orm import sessionmaker


sys.path.append(os.path.abspath(os.path.join('..')))

In [18]:
load_dotenv()  # Carga las variables del archivo .env

config = {
    'host': os.getenv('HOST'),
    'user': os.getenv('USER'),
    'password': os.getenv('PASSWORD'),
    'database': os.getenv('DATABASE'),
    'port': os.getenv('PORT')
}

In [19]:
# Crear la URL de conexión para SQLAlchemy
db_url = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/caso_7"

# Crear el motor de SQLAlchemy
engine = create_engine(db_url, echo=True)



In [20]:
Base = declarative_base()


# **1. GENERACIÓN DE TABLAS**

In [21]:
class ProductHierarchy(Base):

    __tablename__ = 'product_hierarchy'
    __table_args__= {'extend_existing':True}

    #| ------------------ Columnas ----------------|

    id =Column(Integer, primary_key=True)
    parent_id = Column(Integer)
    level_text = Column(VARCHAR(19))
    level_name = Column(VARCHAR(8))


class ProductPrices(Base):

    __tablename__ = 'product_prices'
    __table_args__= {'extend_existing':True}

    #| ------------------ Columnas ----------------|
    
    id =Column(Integer, primary_key=True)
    product_id = Column(VARCHAR(6))
    price = Integer

class ProductDetails(Base):

    __tablename__ = 'product_details'
    __table_args__= {'extend_existing':True}
    
    #| ------------------ Columnas ----------------|

    product_id = Column(VARCHAR(6), primary_key=True)
    price = Column(Integer)
    product_name = Column(VARCHAR(32))
    category_id = Column(Integer)
    segment_id = Column(Integer)
    style_id = Column(Integer)
    category_name = Column(VARCHAR(6))
    segment_name =Column(VARCHAR(6))
    style_name = Column(VARCHAR(19))


class Sales(Base):

    __tablename__ = 'sales'
    __table_args__= {'extend_existing':True}

 #| ------------------ Columnas ----------------|
    id = Column(Integer,  primary_key=True, autoincrement=True)
    prod_id = Column(VARCHAR(6))
    qty = Column(Integer)
    price = Column(Integer)
    discount = Column(Integer)
    member = Column(Boolean)
    txn_id =Column(VARCHAR(32))
    start_txn_time = Column(TIMESTAMP(timezone=True))

Base.metadata.create_all(engine)


2025-05-02 13:40:31,040 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-05-02 13:40:31,041 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:31,043 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-05-02 13:40:31,043 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:31,044 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-05-02 13:40:31,044 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:31,046 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:31,047 INFO sqlalchemy.engine.Engine DESCRIBE `caso_7`.`product_hierarchy`
2025-05-02 13:40:31,048 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:31,051 INFO sqlalchemy.engine.Engine DESCRIBE `caso_7`.`product_prices`
2025-05-02 13:40:31,052 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:31,054 INFO sqlalchemy.engine.Engine DESCRIBE `caso_7`.`product_details`
2025-05-02 13:40:31,056 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:31,057 IN

## **1.1 CARGANDO LOS DATOS**

In [22]:
from utils import funciones_utiles

product_hierarchy_data = funciones_utiles.load_text_as_tuples('datos_tablas/product_hierarchy_data.txt')
product_prices_data = funciones_utiles.load_text_as_tuples('datos_tablas/product_prices_data.txt')
product_details_data =funciones_utiles.load_text_as_tuples('datos_tablas/product_details_data.txt')
sales_data = funciones_utiles.load_text_as_tuples('datos_tablas/sales_data.txt')


In [23]:
# Creando la session
Session = sessionmaker(bind=engine)
session = Session()

In [24]:
product_hierarchy_data

[('1', None, 'Womens', 'Category'),
 ('2', None, 'Mens', 'Category'),
 ('3', '1', 'Jeans', 'Segment'),
 ('4', '1', 'Jacket', 'Segment'),
 ('5', '2', 'Shirt', 'Segment'),
 ('6', '2', 'Socks', 'Segment'),
 ('7', '3', 'Navy Oversized', 'Style'),
 ('8', '3', 'Black Straight', 'Style'),
 ('9', '3', 'Cream Relaxed', 'Style'),
 ('10', '4', 'Khaki Suit', 'Style'),
 ('11', '4', 'Indigo Rain', 'Style'),
 ('12', '4', 'Grey Fashion', 'Style'),
 ('13', '5', 'White Tee', 'Style'),
 ('14', '5', 'Teal Button Up', 'Style'),
 ('15', '5', 'Blue Polo', 'Style'),
 ('16', '6', 'Navy Solid', 'Style'),
 ('17', '6', 'White Striped', 'Style'),
 ('18', '6', 'Pink Fluro Polkadot', 'Style')]

### **1.1.1 Cargando datos `product_hierarchy`**

In [25]:

objects = [
    ProductHierarchy(

        id =int(id),
        parent_id = int(parent_id) if parent_id is not None else None,
        level_text = level_text,
        level_name = level_name
    )
    for id, parent_id, level_text, level_name in product_hierarchy_data
]

session.bulk_save_objects(objects)
session.commit()

2025-05-02 13:40:31,460 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:31,462 INFO sqlalchemy.engine.Engine INSERT INTO product_hierarchy (id, level_text, level_name) VALUES (%(id)s, %(level_text)s, %(level_name)s)
2025-05-02 13:40:31,463 INFO sqlalchemy.engine.Engine [generated in 0.00126s] [{'id': 1, 'level_text': 'Womens', 'level_name': 'Category'}, {'id': 2, 'level_text': 'Mens', 'level_name': 'Category'}]
2025-05-02 13:40:31,465 INFO sqlalchemy.engine.Engine INSERT INTO product_hierarchy (id, parent_id, level_text, level_name) VALUES (%(id)s, %(parent_id)s, %(level_text)s, %(level_name)s)
2025-05-02 13:40:31,466 INFO sqlalchemy.engine.Engine [generated in 0.00068s] [{'id': 3, 'parent_id': 1, 'level_text': 'Jeans', 'level_name': 'Segment'}, {'id': 4, 'parent_id': 1, 'level_text': 'Jacket', 'level_name': 'Segment'}, {'id': 5, 'parent_id': 2, 'level_text': 'Shirt', 'level_name': 'Segment'}, {'id': 6, 'parent_id': 2, 'level_text': 'Socks', 'level_name': 'Segment'}, {'

### **1.1.2 Cargando datos `ProductPrices`**

In [26]:

objects = [
    ProductPrices(

        id =int(id),
        product_id = product_id,
        price = int(price)
    )
    for id, product_id, price in product_prices_data
]

session.bulk_save_objects(objects)
session.commit()

2025-05-02 13:40:31,487 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:31,488 INFO sqlalchemy.engine.Engine INSERT INTO product_prices (id, product_id) VALUES (%(id)s, %(product_id)s)
2025-05-02 13:40:31,489 INFO sqlalchemy.engine.Engine [generated in 0.00074s] [{'id': 7, 'product_id': 'c4a632'}, {'id': 8, 'product_id': 'e83aa3'}, {'id': 9, 'product_id': 'e31d39'}, {'id': 10, 'product_id': 'd5e9a6'}, {'id': 11, 'product_id': '72f5d4'}, {'id': 12, 'product_id': '9ec847'}, {'id': 13, 'product_id': '5d267b'}, {'id': 14, 'product_id': 'c8d436'}  ... displaying 10 of 12 total bound parameter sets ...  {'id': 17, 'product_id': 'b9a74d'}, {'id': 18, 'product_id': '2feb6b'}]
2025-05-02 13:40:31,491 INFO sqlalchemy.engine.Engine COMMIT


### **1.1.3 Cargando datos `ProductDetails`**

In [27]:
objects = [
    ProductDetails(

        product_id = product_id, 
        price = int(price),
        product_name = product_name,
        category_id = int(category_id),
        segment_id = int(segment_id),
        style_id = int(style_id),
        category_name = category_name,
        segment_name =segment_name,
        style_name = style_name,
    )
    for product_id, price, product_name, category_id, segment_id,  style_id, category_name,  segment_name, style_name in product_details_data
]

session.bulk_save_objects(objects)
session.commit()

2025-05-02 13:40:31,519 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:31,521 INFO sqlalchemy.engine.Engine INSERT INTO product_details (product_id, price, product_name, category_id, segment_id, style_id, category_name, segment_name, style_name) VALUES (%(product_id)s, %(price)s, %(product_name)s, %(category_id)s, %(segment_id)s, %(style_id)s, %(category_name)s, %(segment_name)s, %(style_name)s)
2025-05-02 13:40:31,522 INFO sqlalchemy.engine.Engine [generated in 0.00150s] [{'product_id': 'c4a632', 'price': 13, 'product_name': 'Navy Oversized Jeans - Womens', 'category_id': 1, 'segment_id': 3, 'style_id': 7, 'category_name': 'Womens', 'segment_name': 'Jeans', 'style_name': 'Navy Oversized'}, {'product_id': 'e83aa3', 'price': 32, 'product_name': 'Black Straight Jeans - Womens', 'category_id': 1, 'segment_id': 3, 'style_id': 8, 'category_name': 'Womens', 'segment_name': 'Jeans', 'style_name': 'Black Straight'}, {'product_id': 'e31d39', 'price': 10, 'product_name': 'Cream 

### **1.1.3 Cargando datos `Sales`**

In [28]:
objects = [
    Sales(

        prod_id = prod_id,
        qty = int(qty),
        price = int(price),
        discount = int(discount),
        member =True if member == 't' else False,
        txn_id =txn_id,
        start_txn_time = start_txn_time,
    )
    for prod_id, qty, price, discount, member, txn_id,  start_txn_time in sales_data
]

session.bulk_save_objects(objects)
session.commit()

2025-05-02 13:40:32,000 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:32,115 INFO sqlalchemy.engine.Engine INSERT INTO sales (prod_id, qty, price, discount, `member`, txn_id, start_txn_time) VALUES (%(prod_id)s, %(qty)s, %(price)s, %(discount)s, %(member)s, %(txn_id)s, %(start_txn_time)s)
2025-05-02 13:40:32,116 INFO sqlalchemy.engine.Engine [generated in 0.05252s] [{'prod_id': 'c4a632', 'qty': 4, 'price': 13, 'discount': 17, 'member': 1, 'txn_id': '54f307', 'start_txn_time': '2021-02-13 01:59:43.296'}, {'prod_id': '5d267b', 'qty': 4, 'price': 40, 'discount': 17, 'member': 1, 'txn_id': '54f307', 'start_txn_time': '2021-02-13 01:59:43.296'}, {'prod_id': 'b9a74d', 'qty': 4, 'price': 17, 'discount': 17, 'member': 1, 'txn_id': '54f307', 'start_txn_time': '2021-02-13 01:59:43.296'}, {'prod_id': '2feb6b', 'qty': 2, 'price': 29, 'discount': 17, 'member': 1, 'txn_id': '54f307', 'start_txn_time': '2021-02-13 01:59:43.296'}, {'prod_id': 'c4a632', 'qty': 5, 'price': 13, 'discoun

<center><b><h1>PREGUNTAS</h1></b></center>

## **Análisis de Ventas de Alto Nivel**


# **1. ¿Cuál fue la cantidad total vendida de todos los productos?**

In [29]:
query = '''

    SELECT
        d.product_name,
        SUM(s.qty) AS total_vendida
    FROM 
        sales s
    INNER JOIN
        product_details d
        ON
        s.prod_id = d.product_id
    GROUP BY
        d.product_name
    ORDER BY
        total_vendida DESC

'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:40:32,556 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:32,557 INFO sqlalchemy.engine.Engine 

    SELECT
        d.product_name,
        SUM(s.qty) AS total_vendida
    FROM 
        sales s
    INNER JOIN
        product_details d
        ON
        s.prod_id = d.product_id
    GROUP BY
        d.product_name
    ORDER BY
        total_vendida DESC


2025-05-02 13:40:32,558 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:32,598 INFO sqlalchemy.engine.Engine ROLLBACK


,product_name,total_vendida
0,Grey Fashion Jacket - Womens,3876.0
1,Navy Oversized Jeans - Womens,3856.0
2,Blue Polo Shirt - Mens,3819.0
3,White Tee Shirt - Mens,3800.0
4,Navy Solid Socks - Mens,3792.0
5,Black Straight Jeans - Womens,3786.0
6,Pink Fluro Polkadot Socks - Mens,3770.0
7,Indigo Rain Jacket - Womens,3757.0
8,Khaki Suit Jacket - Womens,3752.0
9,Cream Relaxed Jeans - Womens,3707.0


# **2. ¿Cuál es el ingreso total generado por todos los productos antes de los descuentos?**


In [30]:
query = '''

    SELECT
        d.product_name,
        SUM(s.qty) * SUM(s.price) as total_ingresos
    FROM 
        sales s
    INNER JOIN
        product_details d
        ON
        s.prod_id = d.product_id
    GROUP BY
        d.product_name
   

'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:40:32,620 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:32,621 INFO sqlalchemy.engine.Engine 

    SELECT
        d.product_name,
        SUM(s.qty) * SUM(s.price) as total_ingresos
    FROM 
        sales s
    INNER JOIN
        product_details d
        ON
        s.prod_id = d.product_id
    GROUP BY
        d.product_name
   


2025-05-02 13:40:32,622 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:32,664 INFO sqlalchemy.engine.Engine ROLLBACK


,product_name,total_ingresos
0,Navy Oversized Jeans - Womens,63863072.0
1,White Tee Shirt - Mens,192736000.0
2,White Striped Socks - Mens,77233805.0
3,Pink Fluro Polkadot Socks - Mens,137537140.0
4,Cream Relaxed Jeans - Womens,46078010.0
5,Indigo Rain Jacket - Womens,89228750.0
6,Blue Polo Shirt - Mens,276022044.0
7,Navy Solid Socks - Mens,174871872.0
8,Black Straight Jeans - Womens,150955392.0
9,Khaki Suit Jacket - Womens,107611112.0


# **3.¿Cuál fue el monto total de descuento para todos los productos?**


In [31]:
query = '''

    SELECT
        d.product_name,
        SUM(s.qty * s.price * s.discount/100) as total_descuentos
    FROM 
        sales s
    INNER JOIN
        product_details d
        ON
        s.prod_id = d.product_id
    GROUP BY
        d.product_name
   

'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:40:32,682 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:32,683 INFO sqlalchemy.engine.Engine 

    SELECT
        d.product_name,
        SUM(s.qty * s.price * s.discount/100) as total_descuentos
    FROM 
        sales s
    INNER JOIN
        product_details d
        ON
        s.prod_id = d.product_id
    GROUP BY
        d.product_name
   


2025-05-02 13:40:32,684 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:32,734 INFO sqlalchemy.engine.Engine ROLLBACK


,product_name,total_descuentos
0,Navy Oversized Jeans - Womens,6135.61
1,White Tee Shirt - Mens,18377.60
2,White Striped Socks - Mens,7410.81
3,Pink Fluro Polkadot Socks - Mens,12952.27
4,Cream Relaxed Jeans - Womens,4463.40
5,Indigo Rain Jacket - Womens,8642.53
6,Blue Polo Shirt - Mens,26819.07
7,Navy Solid Socks - Mens,16650.36
8,Black Straight Jeans - Womens,14744.96
9,Khaki Suit Jacket - Womens,10243.05


# Análisis de transacciones 

# **1. ¿Cuántas transacciones únicas hubo?**


In [32]:
query = '''

    SELECT
       COUNT(DISTINCT txn_id) AS transacciones_unicas
    FROM 
        sales

   

'''


pd.read_sql_query(query, con=engine)




2025-05-02 13:40:32,758 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:32,759 INFO sqlalchemy.engine.Engine 

    SELECT
       COUNT(DISTINCT txn_id) AS transacciones_unicas
    FROM 
        sales

   


2025-05-02 13:40:32,760 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:32,773 INFO sqlalchemy.engine.Engine ROLLBACK


,transacciones_unicas
0,2500


# **2.-¿Cuál es el promedio de productos únicos comprados en cada transacción?**

In [33]:
query = '''

    WITH cuenta_prod_unicos AS (
    SELECT
       COUNT(DISTINCT prod_id) AS cuenta_productos_unicos
    FROM 
        sales
    GROUP BY 
        txn_id )

    SELECT
        AVG(cuenta_productos_unicos)

    FROM
        cuenta_prod_unicos

'''


pd.read_sql_query(query, con=engine)




2025-05-02 13:40:32,791 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:32,792 INFO sqlalchemy.engine.Engine 

    WITH cuenta_prod_unicos AS (
    SELECT
       COUNT(DISTINCT prod_id) AS cuenta_productos_unicos
    FROM 
        sales
    GROUP BY 
        txn_id )

    SELECT
        AVG(cuenta_productos_unicos)

    FROM
        cuenta_prod_unicos


2025-05-02 13:40:32,794 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:32,818 INFO sqlalchemy.engine.Engine ROLLBACK


,AVG(cuenta_productos_unicos)
0,6.038


# **3.- ¿Cuáles son los valores del percentil 25, 50 y 75 para los ingresos por transacción?**


In [34]:
query = '''

WITH total_ventas AS (
    SELECT
        txn_id,
        SUM(qty * price) AS total_sales
    FROM
        sales
    GROUP BY
        txn_id
),
ventas_rankeadas AS (
    SELECT
        total_sales,
        ROW_NUMBER() OVER (ORDER BY total_sales) AS row_num,
        COUNT(*) OVER () AS total_count
    FROM total_ventas
)
SELECT
    ROUND(AVG(CASE WHEN row_num = CEIL(0.25 * total_count) THEN total_sales END), 2) AS percentile_25,
    ROUND(AVG(CASE WHEN row_num = CEIL(0.50 * total_count) THEN total_sales END), 2) AS percentile_50,
    ROUND(AVG(CASE WHEN row_num = CEIL(0.75 * total_count) THEN total_sales END), 2) AS percentile_75
FROM ventas_rankeadas;

'''


pd.read_sql_query(query, con=engine)




2025-05-02 13:40:32,837 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:32,838 INFO sqlalchemy.engine.Engine 

WITH total_ventas AS (
    SELECT
        txn_id,
        SUM(qty * price) AS total_sales
    FROM
        sales
    GROUP BY
        txn_id
),
ventas_rankeadas AS (
    SELECT
        total_sales,
        ROW_NUMBER() OVER (ORDER BY total_sales) AS row_num,
        COUNT(*) OVER () AS total_count
    FROM total_ventas
)
SELECT
    ROUND(AVG(CASE WHEN row_num = CEIL(0.25 * total_count) THEN total_sales END), 2) AS percentile_25,
    ROUND(AVG(CASE WHEN row_num = CEIL(0.50 * total_count) THEN total_sales END), 2) AS percentile_50,
    ROUND(AVG(CASE WHEN row_num = CEIL(0.75 * total_count) THEN total_sales END), 2) AS percentile_75
FROM ventas_rankeadas;


2025-05-02 13:40:32,839 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:32,865 INFO sqlalchemy.engine.Engine ROLLBACK


,percentile_25,percentile_50,percentile_75
0,375.0,509.0,647.0


 # **4. ¿Cuál es el valor promedio del descuento por transacción?**


In [35]:
query = '''
WITH total_descuento_por_transaccion AS (
SELECT
    txn_id, 
    SUM(qty * price * discount/100) as total_descuentos

FROM
    sales
GROUP BY
    txn_id )

SELECT
    ROUND(AVG(total_descuentos)) AS promedio_dscto_transaccion
FROM 
    total_descuento_por_transaccion

'''


pd.read_sql_query(query, con=engine)




2025-05-02 13:40:32,885 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:32,886 INFO sqlalchemy.engine.Engine 
WITH total_descuento_por_transaccion AS (
SELECT
    txn_id, 
    SUM(qty * price * discount/100) as total_descuentos

FROM
    sales
GROUP BY
    txn_id )

SELECT
    ROUND(AVG(total_descuentos)) AS promedio_dscto_transaccion
FROM 
    total_descuento_por_transaccion


2025-05-02 13:40:32,887 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:32,910 INFO sqlalchemy.engine.Engine ROLLBACK


,promedio_dscto_transaccion
0,62.0


#  **5. ¿Cuál es el porcentaje de división de todas las transacciones entre miembros y no miembros?**


In [36]:
query = '''


    SELECT
       member,
       COUNT(DISTINCT txn_id)*100.0 / (SELECT COUNT(DISTINCT txn_id) FROM sales) AS total_transacciones

    FROM 
        sales
    GROUP BY
        member

   
'''


pd.read_sql_query(query, con=engine)




2025-05-02 13:40:32,933 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:32,934 INFO sqlalchemy.engine.Engine 


    SELECT
       member,
       COUNT(DISTINCT txn_id)*100.0 / (SELECT COUNT(DISTINCT txn_id) FROM sales) AS total_transacciones

    FROM 
        sales
    GROUP BY
        member

   

2025-05-02 13:40:32,934 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:32,973 INFO sqlalchemy.engine.Engine ROLLBACK


,member,total_transacciones
0,0,39.8
1,1,60.2


# **6. ¿Cuál es el ingreso promedio para las transacciones de miembros y las transacciones de no miembros?**

In [37]:
query = '''


    WITH total_sales_ AS (
    SELECT
       member,
       txn_id,
       SUM(qty * price) AS total_sales

    FROM 
        sales
    GROUP BY
        member, txn_id )

    SELECT
        AVG(total_sales) as ingreso_promedio_por_transaccion
    FROM
        total_sales_
    GROUP BY
        member

   
'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:40:32,997 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:32,998 INFO sqlalchemy.engine.Engine 


    WITH total_sales_ AS (
    SELECT
       member,
       txn_id,
       SUM(qty * price) AS total_sales

    FROM 
        sales
    GROUP BY
        member, txn_id )

    SELECT
        AVG(total_sales) as ingreso_promedio_por_transaccion
    FROM
        total_sales_
    GROUP BY
        member

   

2025-05-02 13:40:32,999 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:33,026 INFO sqlalchemy.engine.Engine ROLLBACK


,ingreso_promedio_por_transaccion
0,516.2684
1,515.0442


# **Análisis de Productos** 

# **1. ¿Cuáles son los 3 productos principales según el ingreso total antes del descuento?**


In [38]:
query = '''

    WITH total_ingresos_antes_descuento AS (
    SELECT
        p.product_id, 
        p.product_name,
        SUM(s.qty * s.price) AS total_ingresos_bf_descuento
    FROM
        sales s
    INNER JOIN 
        product_details p
    ON s.prod_id = p.product_id
    GROUP BY
        p.product_id, 
        p.product_name
    ORDER BY
        total_ingresos_bf_descuento DESC ),

    ranking_ingresos_antes_descuento AS (

        SELECT
            product_id,
            product_name,
            total_ingresos_bf_descuento,
            ROW_NUMBER() OVER(ORDER BY total_ingresos_bf_descuento DESC) as top
        FROM 
           total_ingresos_antes_descuento )
    

    SELECT
        *
    FROM
        ranking_ingresos_antes_descuento
    WHERE 
        top <=3
   
'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:40:33,046 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:33,047 INFO sqlalchemy.engine.Engine 

    WITH total_ingresos_antes_descuento AS (
    SELECT
        p.product_id, 
        p.product_name,
        SUM(s.qty * s.price) AS total_ingresos_bf_descuento
    FROM
        sales s
    INNER JOIN 
        product_details p
    ON s.prod_id = p.product_id
    GROUP BY
        p.product_id, 
        p.product_name
    ORDER BY
        total_ingresos_bf_descuento DESC ),

    ranking_ingresos_antes_descuento AS (

        SELECT
            product_id,
            product_name,
            total_ingresos_bf_descuento,
            ROW_NUMBER() OVER(ORDER BY total_ingresos_bf_descuento DESC) as top
        FROM 
           total_ingresos_antes_descuento )
    

    SELECT
        *
    FROM
        ranking_ingresos_antes_descuento
    WHERE 
        top <=3
   

2025-05-02 13:40:33,048 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:33,096 INFO sq

,product_id,product_name,total_ingresos_bf_descuento,top
0,2a2353,Blue Polo Shirt - Mens,217683.0,1
1,9ec847,Grey Fashion Jacket - Womens,209304.0,2
2,5d267b,White Tee Shirt - Mens,152000.0,3


# **2.- ¿Cuál es la cantidad total, ingreso y descuento para cada segmento?**


In [39]:
query = '''

  
    SELECT
        p.segment_name,
        SUM(s.qty) AS cantidad_total,
        SUM(s.qty*s.price) AS total_ingresos,
        SUM(s.qty * s.price * COALESCE(s.discount, 0) / 100) AS total_descuentos

     FROM
        sales s
    INNER JOIN 
        product_details p
    ON s.prod_id = p.product_id
    GROUP BY
        p.segment_name
   
'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:40:33,123 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:33,124 INFO sqlalchemy.engine.Engine 

  
    SELECT
        p.segment_name,
        SUM(s.qty) AS cantidad_total,
        SUM(s.qty*s.price) AS total_ingresos,
        SUM(s.qty * s.price * COALESCE(s.discount, 0) / 100) AS total_descuentos

     FROM
        sales s
    INNER JOIN 
        product_details p
    ON s.prod_id = p.product_id
    GROUP BY
        p.segment_name
   

2025-05-02 13:40:33,125 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:33,172 INFO sqlalchemy.engine.Engine ROLLBACK


,segment_name,cantidad_total,total_ingresos,total_descuentos
0,Jeans,11349.0,208350.0,25343.97
1,Shirt,11265.0,406143.0,49594.27
2,Socks,11217.0,307977.0,37013.44
3,Jacket,11385.0,366983.0,44277.46


# **3.- ¿Cuál es el producto más vendido para cada segmento?**


In [40]:
query = '''

    WITH cuenta_sales_por_segmento AS (
    SELECT
        p.segment_name,
        p.product_name,
        SUM(s.qty) AS cantidad_total_vendidas

     FROM
        sales s
    INNER JOIN 
        product_details p
    ON s.prod_id = p.product_id
    GROUP BY
        p.segment_name,
        p.product_name ),

    top_por_seccion AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY segment_name ORDER BY cantidad_total_vendidas DESC) as top_sections
    FROM
        cuenta_sales_por_segmento )

    SELECT
        *
    FROM 
        top_por_seccion
    WHERE
        top_sections =1
    ORDER BY
        cantidad_total_vendidas DESC
   
'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:40:33,203 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:33,204 INFO sqlalchemy.engine.Engine 

    WITH cuenta_sales_por_segmento AS (
    SELECT
        p.segment_name,
        p.product_name,
        SUM(s.qty) AS cantidad_total_vendidas

     FROM
        sales s
    INNER JOIN 
        product_details p
    ON s.prod_id = p.product_id
    GROUP BY
        p.segment_name,
        p.product_name ),

    top_por_seccion AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY segment_name ORDER BY cantidad_total_vendidas DESC) as top_sections
    FROM
        cuenta_sales_por_segmento )

    SELECT
        *
    FROM 
        top_por_seccion
    WHERE
        top_sections =1
    ORDER BY
        cantidad_total_vendidas DESC
   

2025-05-02 13:40:33,205 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:33,252 INFO sqlalchemy.engine.Engine ROLLBACK


,segment_name,product_name,cantidad_total_vendidas,top_sections
0,Jacket,Grey Fashion Jacket - Womens,3876.0,1
1,Jeans,Navy Oversized Jeans - Womens,3856.0,1
2,Shirt,Blue Polo Shirt - Mens,3819.0,1
3,Socks,Navy Solid Socks - Mens,3792.0,1


# **4.- ¿Cuál es la cantidad total, ingreso y descuento para cada categoría?**


In [41]:
query = '''

  
    SELECT
        p.category_name,
        SUM(s.qty) AS cantidad_total,
        SUM(s.qty*s.price) AS total_ingresos,
        SUM(s.qty * s.price * COALESCE(s.discount, 0) / 100) AS total_descuentos

     FROM
        sales s
    INNER JOIN 
        product_details p
    ON s.prod_id = p.product_id
    GROUP BY
        p.category_name
   
'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:40:33,280 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:33,281 INFO sqlalchemy.engine.Engine 

  
    SELECT
        p.category_name,
        SUM(s.qty) AS cantidad_total,
        SUM(s.qty*s.price) AS total_ingresos,
        SUM(s.qty * s.price * COALESCE(s.discount, 0) / 100) AS total_descuentos

     FROM
        sales s
    INNER JOIN 
        product_details p
    ON s.prod_id = p.product_id
    GROUP BY
        p.category_name
   

2025-05-02 13:40:33,282 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:40:33,335 INFO sqlalchemy.engine.Engine ROLLBACK


,category_name,cantidad_total,total_ingresos,total_descuentos
0,Womens,22734.0,575333.0,69621.43
1,Mens,22482.0,714120.0,86607.71


# **5. ¿Cuál es el producto más vendido para cada categoría?**


In [42]:
query = '''

    WITH ventas_por_producto_categoria AS (
        SELECT
            p.category_name,
            p.product_id,
            p.product_name,
            SUM(s.qty) AS cantidad_total_vendidas
        FROM 
            sales s
        INNER JOIN 
            product_details p ON s.prod_id = p.product_id
        GROUP BY 
            p.category_name, 
            p.product_id, 
            p.product_name
        ),
    ranking_por_categoria AS (
        SELECT
            *,
            ROW_NUMBER() OVER (
                PARTITION BY category_name 
                ORDER BY cantidad_total_vendidas DESC
            ) AS rank_categoria
        FROM ventas_por_producto_categoria
    )
    SELECT
        category_name,
        product_name,
        cantidad_total_vendidas
    FROM 
        ranking_por_categoria
    WHERE 
        rank_categoria = 1
    ORDER BY 
        cantidad_total_vendidas DESC;
   
'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:40:33,360 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:33,360 INFO sqlalchemy.engine.Engine 

    WITH ventas_por_producto_categoria AS (
        SELECT
            p.category_name,
            p.product_id,
            p.product_name,
            SUM(s.qty) AS cantidad_total_vendidas
        FROM 
            sales s
        INNER JOIN 
            product_details p ON s.prod_id = p.product_id
        GROUP BY 
            p.category_name, 
            p.product_id, 
            p.product_name
        ),
    ranking_por_categoria AS (
        SELECT
            *,
            ROW_NUMBER() OVER (
                PARTITION BY category_name 
                ORDER BY cantidad_total_vendidas DESC
            ) AS rank_categoria
        FROM ventas_por_producto_categoria
    )
    SELECT
        category_name,
        product_name,
        cantidad_total_vendidas
    FROM 
        ranking_por_categoria
    WHERE 
        rank_categoria = 1
    ORDER BY 
    

,category_name,product_name,cantidad_total_vendidas
0,Womens,Grey Fashion Jacket - Womens,3876.0
1,Mens,Blue Polo Shirt - Mens,3819.0


# **6. ¿Cuál es la distribución porcentual del ingreso por producto para cada segmento?** 


In [43]:
query = '''

    WITH ingresos_prod_segmento AS (
        SELECT
            p.segment_name,
            p.product_name,
            SUM(s.qty*s.price) AS total_ingresos

        FROM
            sales s
        INNER JOIN 
            product_details p
        ON s.prod_id = p.product_id
        GROUP BY
            p.segment_name,
            p.product_name ),
    total_segmento AS (
        SELECT
            segment_name,
            product_name,
            total_ingresos,
            SUM(total_ingresos) OVER (PARTITION BY segment_name) AS total_segmento
        FROM
            ingresos_prod_segmento
    )

    SELECT
        segment_name,
        product_name,
        total_ingresos,
        ROUND((total_ingresos*100.0 / total_segmento ), 2) as pctje
    FROM
        total_segmento

    
   
'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:40:33,438 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:40:33,439 INFO sqlalchemy.engine.Engine 

    WITH ingresos_prod_segmento AS (
        SELECT
            p.segment_name,
            p.product_name,
            SUM(s.qty*s.price) AS total_ingresos

        FROM
            sales s
        INNER JOIN 
            product_details p
        ON s.prod_id = p.product_id
        GROUP BY
            p.segment_name,
            p.product_name ),
    total_segmento AS (
        SELECT
            segment_name,
            product_name,
            total_ingresos,
            SUM(total_ingresos) OVER (PARTITION BY segment_name) AS total_segmento
        FROM
            ingresos_prod_segmento
    )

    SELECT
        segment_name,
        product_name,
        total_ingresos,
        ROUND((total_ingresos*100.0 / total_segmento ), 2) as pctje
    FROM
        total_segmento

    
   

2025-05-02 13:40:33,439 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05

,segment_name,product_name,total_ingresos,pctje
0,Jacket,Indigo Rain Jacket - Womens,71383.0,19.45
1,Jacket,Khaki Suit Jacket - Womens,86296.0,23.51
2,Jacket,Grey Fashion Jacket - Womens,209304.0,57.03
3,Jeans,Navy Oversized Jeans - Womens,50128.0,24.06
4,Jeans,Cream Relaxed Jeans - Womens,37070.0,17.79
5,Jeans,Black Straight Jeans - Womens,121152.0,58.15
6,Shirt,White Tee Shirt - Mens,152000.0,37.43
7,Shirt,Blue Polo Shirt - Mens,217683.0,53.60
8,Shirt,Teal Button Up Shirt - Mens,36460.0,8.98
9,Socks,White Striped Socks - Mens,62135.0,20.18


# **7.- ¿Cuál es la distribución porcentual del ingreso por segmento para cada categoría?**


In [ ]:
query = '''

    WITH ingresos_segmento_category AS (
        SELECT
            p.segment_name,
            p.category_name,
            SUM(s.qty*s.price) AS total_ingresos

        FROM
            sales s
        INNER JOIN 
            product_details p
        ON s.prod_id = p.product_id
        GROUP BY
            p.segment_name,
            p.category_name ),
    total_segmento AS (
        SELECT
            segment_name,
            category_name,
            total_ingresos,
            SUM(total_ingresos) OVER (PARTITION BY category_name) AS total_segmento
        FROM
            ingresos_segmento_category
    )

    SELECT
        segment_name,
        category_name,
        total_ingresos,
        ROUND((total_ingresos*100.0 / total_segmento ), 2) as pctje
    FROM
        total_segmento

    
   
'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:43:02,239 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:43:02,240 INFO sqlalchemy.engine.Engine 

    WITH ingresos_prod_segmento AS (
        SELECT
            p.segment_name,
            p.category_name,
            SUM(s.qty*s.price) AS total_ingresos

        FROM
            sales s
        INNER JOIN 
            product_details p
        ON s.prod_id = p.product_id
        GROUP BY
            p.segment_name,
            p.category_name ),
    total_segmento AS (
        SELECT
            segment_name,
            category_name,
            total_ingresos,
            SUM(total_ingresos) OVER (PARTITION BY category_name) AS total_segmento
        FROM
            ingresos_prod_segmento
    )

    SELECT
        segment_name,
        category_name,
        total_ingresos,
        ROUND((total_ingresos*100.0 / total_segmento ), 2) as pctje
    FROM
        total_segmento

    
   

2025-05-02 13:43:02,241 INFO sqlalchemy.engine.Engine [raw sql] {}
20

,segment_name,category_name,total_ingresos,pctje
0,Shirt,Mens,406143.0,56.87
1,Socks,Mens,307977.0,43.13
2,Jeans,Womens,208350.0,36.21
3,Jacket,Womens,366983.0,63.79


# **8. ¿Cuál es la distribución porcentual del ingreso total por categoría?**


In [ ]:
query = '''

    WITH ingresos_por_categoria AS (
        SELECT
            p.category_name,
            SUM(s.qty*s.price) AS total_ingresos

        FROM
            sales s
        INNER JOIN 
            product_details p
        ON s.prod_id = p.product_id
        GROUP BY
            p.category_name ),
    total_segmento AS (
        SELECT
            category_name,
            total_ingresos
        FROM
            ingresos_por_categoria 
    )

    SELECT
        category_name,
        total_ingresos,
        ROUND((total_ingresos*100.0 / (SELECT SUM(total_ingresos)FROM total_segmento) ), 2) as pctje
    FROM
        total_segmento

    
   
'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:52:44,848 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:52:44,849 INFO sqlalchemy.engine.Engine 

    WITH ingresos_segmento_category AS (
        SELECT
            p.category_name,
            SUM(s.qty*s.price) AS total_ingresos

        FROM
            sales s
        INNER JOIN 
            product_details p
        ON s.prod_id = p.product_id
        GROUP BY
            p.category_name ),
    total_segmento AS (
        SELECT
            category_name,
            total_ingresos
        FROM
            ingresos_segmento_category 
    )

    SELECT
        category_name,
        total_ingresos,
        ROUND((total_ingresos*100.0 / (SELECT SUM(total_ingresos)FROM total_segmento) ), 2) as pctje
    FROM
        total_segmento

    
   

2025-05-02 13:52:44,850 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-02 13:52:44,886 INFO sqlalchemy.engine.Engine ROLLBACK


,category_name,total_ingresos,pctje
0,Womens,575333.0,44.62
1,Mens,714120.0,55.38



# **9. -¿Cuál es la “penetración” total de transacciones para cada producto?**  
*(pista: penetración = número de transacciones donde se compró al menos 1 unidad de un producto dividido por el número total de transacciones)*

In [67]:
query = '''

    WITH total_transactions AS (

        SELECT
        COUNT(DISTINCT txn_id) AS cuenta_transacciones_totales
    FROM
        sales
    ),
    product_transactions AS (

        SELECT
            prod_id,
            COUNT(DISTINCT txn_id) AS cuenta_transacciones_productos
        FROM
            sales
        GROUP BY
            prod_id
    )

        SELECT
        pt.prod_id,
        pd.product_name,
        pt.cuenta_transacciones_productos,
        tt.cuenta_transacciones_totales,
        CAST(pt.cuenta_transacciones_productos AS REAL) * 100.0 / tt.cuenta_transacciones_totales AS porcentaje_penetracion
    FROM
        product_transactions pt
    CROSS JOIN
        total_transactions tt
    JOIN
        product_details pd ON pt.prod_id = pd.product_id -- Join to get product name
    ORDER BY
        porcentaje_penetracion DESC; -- Optional: Order by penetration
        
   
'''


pd.read_sql_query(query, con=engine)

2025-05-02 13:57:16,831 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 13:57:16,832 INFO sqlalchemy.engine.Engine 

    WITH total_transactions AS (

        SELECT
        COUNT(DISTINCT txn_id) AS cuenta_transacciones_totales
    FROM
        sales
    ),
    product_transactions AS (

        SELECT
            prod_id,
            COUNT(DISTINCT txn_id) AS cuenta_transacciones_productos
        FROM
            sales
        GROUP BY
            prod_id
    )

        SELECT
        pt.prod_id,
        pd.product_name,
        pt.cuenta_transacciones_productos,
        tt.cuenta_transacciones_totales,
        CAST(pt.cuenta_transacciones_productos AS REAL) * 100.0 / tt.cuenta_transacciones_totales AS porcentaje_penetracion
    FROM
        product_transactions pt
    CROSS JOIN
        total_transactions tt
    JOIN
        product_details pd ON pt.prod_id = pd.product_id -- Join to get product name
    ORDER BY
        porcentaje_penetracion DESC; -- Optional: Order by 

,prod_id,product_name,cuenta_transacciones_productos,cuenta_transacciones_totales,porcentaje_penetracion
0,f084eb,Navy Solid Socks - Mens,1281,2500,51.24
1,9ec847,Grey Fashion Jacket - Womens,1275,2500,51.00
2,c4a632,Navy Oversized Jeans - Womens,1274,2500,50.96
3,2a2353,Blue Polo Shirt - Mens,1268,2500,50.72
4,5d267b,White Tee Shirt - Mens,1268,2500,50.72
5,2feb6b,Pink Fluro Polkadot Socks - Mens,1258,2500,50.32
6,72f5d4,Indigo Rain Jacket - Womens,1250,2500,50.00
7,d5e9a6,Khaki Suit Jacket - Womens,1247,2500,49.88
8,e83aa3,Black Straight Jeans - Womens,1246,2500,49.84
9,b9a74d,White Striped Socks - Mens,1243,2500,49.72




# **10. ¿Cuál es la combinación más común de al menos 1 unidad de cualquier 3 productos en una sola transacción?**

In [69]:
query = '''

    WITH TransaccionesConAlMenos3Productos AS (
        SELECT
            txn_id
        FROM
            sales
        GROUP BY
            txn_id
        HAVING
            COUNT(DISTINCT prod_id) >= 3
        ),
    CombinacionesDe3ProductosEnTransacciones AS (
      
        SELECT
            s1.prod_id AS prod_id_1,
            s2.prod_id AS prod_id_2,
            s3.prod_id AS prod_id_3
        FROM
            sales s1
        JOIN
            sales s2 ON s1.txn_id = s2.txn_id
        JOIN
            sales s3 ON s2.txn_id = s3.txn_id
        JOIN
            TransaccionesConAlMenos3Productos t ON s1.txn_id = t.txn_id -- Filtrar solo transacciones con >= 3 productos
        WHERE
            s1.prod_id < s2.prod_id AND s2.prod_id < s3.prod_id
    ),
    ConteoCombinaciones AS (
        SELECT
            prod_id_1,
            prod_id_2,
            prod_id_3,
            COUNT(*) AS frecuencia_combinacion
        FROM
            CombinacionesDe3ProductosEnTransacciones
        GROUP BY
            prod_id_1,
            prod_id_2,
            prod_id_3
    )
    SELECT
        cc.prod_id_1,
        pd1.product_name AS nombre_producto_1,
        cc.prod_id_2,
        pd2.product_name AS nombre_producto_2,
        cc.prod_id_3,
        pd3.product_name AS nombre_producto_3,
        cc.frecuencia_combinacion
    FROM
        ConteoCombinaciones cc
    JOIN
        product_details pd1 ON cc.prod_id_1 = pd1.product_id
    JOIN
        product_details pd2 ON cc.prod_id_2 = pd2.product_id
    JOIN
        product_details pd3 ON cc.prod_id_3 = pd3.product_id
        
   
'''


pd.read_sql_query(query, con=engine)

2025-05-02 14:00:16,102 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 14:00:16,102 INFO sqlalchemy.engine.Engine 

    WITH TransaccionesConAlMenos3Productos AS (
        SELECT
            txn_id
        FROM
            sales
        GROUP BY
            txn_id
        HAVING
            COUNT(DISTINCT prod_id) >= 3
        ),
    CombinacionesDe3ProductosEnTransacciones AS (
      
        SELECT
            s1.prod_id AS prod_id_1,
            s2.prod_id AS prod_id_2,
            s3.prod_id AS prod_id_3
        FROM
            sales s1
        JOIN
            sales s2 ON s1.txn_id = s2.txn_id
        JOIN
            sales s3 ON s2.txn_id = s3.txn_id
        JOIN
            TransaccionesConAlMenos3Productos t ON s1.txn_id = t.txn_id -- Filtrar solo transacciones con >= 3 productos
        WHERE
            s1.prod_id < s2.prod_id AND s2.prod_id < s3.prod_id
    ),
    ConteoCombinaciones AS (
        SELECT
            prod_id_1,
            prod_id_2,
            pr

,prod_id_1,nombre_producto_1,prod_id_2,nombre_producto_2,prod_id_3,nombre_producto_3,frecuencia_combinacion
0,2a2353,Blue Polo Shirt - Mens,2feb6b,Pink Fluro Polkadot Socks - Mens,5d267b,White Tee Shirt - Mens,306
1,2a2353,Blue Polo Shirt - Mens,2feb6b,Pink Fluro Polkadot Socks - Mens,72f5d4,Indigo Rain Jacket - Womens,305
2,2a2353,Blue Polo Shirt - Mens,2feb6b,Pink Fluro Polkadot Socks - Mens,9ec847,Grey Fashion Jacket - Womens,325
3,2a2353,Blue Polo Shirt - Mens,2feb6b,Pink Fluro Polkadot Socks - Mens,b9a74d,White Striped Socks - Mens,307
4,2a2353,Blue Polo Shirt - Mens,2feb6b,Pink Fluro Polkadot Socks - Mens,c4a632,Navy Oversized Jeans - Womens,307
...,...,...,...,...,...,...,...
215,c8d436,Teal Button Up Shirt - Mens,e83aa3,Black Straight Jeans - Womens,f084eb,Navy Solid Socks - Mens,316
216,d5e9a6,Khaki Suit Jacket - Womens,e31d39,Cream Relaxed Jeans - Womens,e83aa3,Black Straight Jeans - Womens,283
217,d5e9a6,Khaki Suit Jacket - Womens,e31d39,Cream Relaxed Jeans - Womens,f084eb,Navy Solid Socks - Mens,325
218,d5e9a6,Khaki Suit Jacket - Womens,e83aa3,Black Straight Jeans - Womens,f084eb,Navy Solid Socks - Mens,311
